In [ ]:
# Cambiar si es necesario

NOMBRE_BASE_DE_DATOS_OPTUNA = 'optimization_lgbm_shock_aguinaldo.db' 
NOMBRE_DE_ESTUDIO_OPTUNA = 'lgbm_cv_shock_aguinaldo' 
ARCHIVO_DATOS_CSV = 'competencia_01_fe_shock_aguinaldo.csv' 
NOMBRE_NOTEBOOK = 'CV_clasico_semillas_shock_aguinaldo'


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time
import pickle

In [ ]:
# Configuración de paths

base_path = os.getcwd()
dataset_path = os.path.join(base_path, 'datos')
modelos_path = os.path.join(base_path, 'modelos')
db_path = os.path.join(base_path, 'db')
dataset_file = ARCHIVO_DATOS_CSV

data = pd.read_csv(os.path.join(dataset_path, dataset_file))

In [ ]:
# Configuración de meses y semillas

meses_train = [202101, 202102, 202103]     # Entrenamiento Optuna con cv clásico (enero, febrero, marzo)
mes_valid = 202104                         # Validación (abril)
mes_test_final = 202106                    # Predicción final (junio)
semillas = [181459, 306491, 336251, 900577, 901751, 182009, 182011, 182027, 182029, 182041]

ganancia_acierto = 780000
costo_estimulo = 20000


In [ ]:
# CONFIGURACIÓN: decidir si eliminar columnas identificatorias

cols_id = ['foto_mes', 'numero_de_cliente']


eliminar_columnas_id = False # Finalmente no se eliminarán

In [ ]:
# Clases y pesos

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)


In [ ]:
# Preparación de datos de entrenamiento para optimización bayesiana con optuna

train_data = data[data['foto_mes'].isin(meses_train)]

cols_to_drop = ['clase_ternaria', 'clase_peso', 'clase_binaria1', 'clase_binaria2']
if eliminar_columnas_id:
    cols_to_drop += cols_id

X_train = train_data.drop(columns=cols_to_drop, errors='ignore')
y_train_binaria1 = train_data['clase_binaria1']
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

print(f"🔧 Columnas eliminadas del entrenamiento: {cols_to_drop}")

🔧 Columnas eliminadas del entrenamiento: ['clase_ternaria', 'clase_peso', 'clase_binaria1', 'clase_binaria2']


In [ ]:
# Función de ganancia personalizada para LightGBM

def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)
    return 'gan_eval', np.max(ganancia), True

In [ ]:
# Objetivo para optuna

def objective(trial):
    # Espacio de búsqueda
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': trial.suggest_int('num_leaves', 8, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.15, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 30), # min_data_in_leaf lo deja en 3
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 5.0),
        'max_depth': trial.suggest_int('max_depth', 3, 40),
        'seed': semillas[0],
        'verbose': -1
    }

    num_boost_round = trial.suggest_int('num_boost_round', 100, 2000)
    y_target = y_train_binaria2

    train_dataset = lgb.Dataset(X_train, label=y_target, weight=w_train)
    nfold = 5

    cv_results = lgb.cv(
        params,
        train_dataset,
        num_boost_round=num_boost_round,
        feval=lgb_gan_eval,
        stratified=True,
        nfold=nfold,
        seed=semillas[0],
        callbacks=[
            lgb.early_stopping(stopping_rounds=int(50 + 5 / params['learning_rate']), verbose=False),
            lgb.log_evaluation(period=200),
        ]
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1
    trial.set_user_attr("best_iter", best_iter)
    
    return max_gan * 5

In [10]:
# Crear carpeta si no existe
os.makedirs(db_path, exist_ok=True)

# Definir base de datos SQLite para guardar resultados
storage_name = "sqlite:///" + os.path.join(db_path, NOMBRE_BASE_DE_DATOS_OPTUNA)

# Definir sampler con 20 iteraciones aleatorias (exploración inicial)
sampler = optuna.samplers.TPESampler(
    n_startup_trials=20,  # 20 iteraciones "no inteligentes"
    seed=semillas[0]
)

# Crear estudio (si ya existe, lo reusa)
study = optuna.create_study(
    direction="maximize",
    study_name=NOMBRE_DE_ESTUDIO_OPTUNA,
    storage=storage_name,
    load_if_exists=True,
    sampler=sampler
)

[I 2025-10-11 22:09:08,438] Using an existing study with name 'lgbm_cv_shock_aguinaldo' instead of creating a new one.


In [11]:
# # Ejecutar optimización: 100 = 20 aleatorias + 80 inteligentes
# study.optimize(objective, n_trials=200)

In [ ]:
# Historial de optimización (trials vs ganancia)

plot_optimization_history(study)


In [ ]:
# Importancia de los HP (da una idea más bien de cuanta variabilidad aporta cada HP). Importante para recortar espacio de búsqueda en futuras optimizaciones

plot_param_importances(study)

In [17]:
# Elegir el trial
trial_number = 113
trial = study.trials[trial_number]

# Recuperar valores
best_params = trial.params
best_value = trial.value
best_iter = trial.user_attrs.get("best_iter", None)

best_target = 'binaria2'

best_params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'verbose': -1
}

best_params.update(trial.params)
best_params.pop('num_boost_round', None) # lightgbm no espera este parámetro en params, si no como argumento aparte en num_boost_round. Pero a ese le pongo el best_iter

print(f"🔎 Trial {trial.number}")
print(f"🎯 Métrica: {best_value}")
print(f"🧪 Best Iter: {best_iter}\n")
print("⚙️ Hiperparámetros:")
for k, v in best_params.items():
    print(f"   {k}: {v}")

🔎 Trial 113
🎯 Métrica: 916960000.0
🧪 Best Iter: 1197

⚙️ Hiperparámetros:
   objective: binary
   boosting_type: gbdt
   first_metric_only: True
   boost_from_average: True
   feature_pre_filter: False
   max_bin: 31
   verbose: -1
   num_leaves: 84
   learning_rate: 0.022781668824537155
   min_data_in_leaf: 19
   feature_fraction: 0.7810058700513337
   bagging_fraction: 0.8694808899659255
   bagging_freq: 7
   lambda_l1: 0.0018940393703961173
   lambda_l2: 1.8064379244551527
   min_gain_to_split: 0.18555044398470172
   max_depth: 20


In [ ]:
# Función de ganancia acumulada con umbral óptimo. Sirve para buscar umbral óptimo del mes de validación (abril)

def mejor_umbral_probabilidad(y_pred, weights):
    """
    Encuentra el umbral de probabilidad óptimo en lugar del N óptimo.
    
    Calcula ganancia usando los pesos para identificar BAJA+2:
    - weights: clase_peso donde 1.00002 = BAJA+2, 1.00001 = BAJA+1, 1.0 = CONTINUA
    - Ganancia NETA por envío:
        * Si acertás BAJA+2: +ganancia_acierto - costo_estimulo
        * Si fallás (BAJA+1 o CONTINUA): -costo_estimulo
    
    Usa las variables globales: ganancia_acierto y costo_estimulo
    
    Returns:
        umbral_optimo: el umbral de probabilidad que maximiza ganancia
        N_en_umbral: cuántos casos quedan por encima de ese umbral
        ganancia_max: la ganancia máxima obtenida
        curva: tupla (ns, ganancias, umbrales) para graficar
    """
    # Filtrar valores finitos
    mask = np.isfinite(y_pred)
    y_pred = np.array(y_pred)[mask]
    weights = np.array(weights)[mask]
    
    # Ordenar por probabilidad descendente
    orden = np.argsort(y_pred)[::-1]
    y_pred_sorted = y_pred[orden]
    weights_sorted = weights[orden]
    
    # Ganancia NETA por envío:
    # - BAJA+2 (peso 1.00002): ganamos ganancia_acierto pero pagamos costo_estimulo
    # - Otros: solo perdemos el costo_estimulo
    ganancias = np.where(
        weights_sorted == 1.00002, 
        ganancia_acierto - costo_estimulo,  # <- usa las variables globales
        -costo_estimulo
    )
    
    gan_acum = np.cumsum(ganancias)
    
    if len(gan_acum) == 0:
        return 0, 0, 0, ([], [], [])
    
    # Buscar máximo en el primer 70% (igual que tu función original)
    limite_busqueda = int(len(gan_acum) * 0.7)
    idx_max = np.argmax(gan_acum[:limite_busqueda])
    
    ganancia_max = gan_acum[idx_max]
    N_optimo = idx_max + 1
    umbral_optimo = y_pred_sorted[idx_max]
    
    # Preparar curva para graficar
    ns = list(range(1, len(gan_acum) + 1))
    umbrales = list(y_pred_sorted)
    
    return umbral_optimo, N_optimo, ganancia_max, (ns, gan_acum, umbrales)

In [ ]:

# EVALUACIÓN MULTISEMILLA CON ENSEMBLE Y UMBRAL DEL ENSEMBLE
# Preparar datos de validación (abril) y test final (junio)

valid_data = data[data['foto_mes'] == mes_valid]
test_data = data[data['foto_mes'] == mes_test_final]

cols_to_drop_eval = ['clase_ternaria', 'clase_peso', 'clase_binaria1', 'clase_binaria2']
if eliminar_columnas_id:
    cols_to_drop_eval += cols_id

X_valid = valid_data.drop(columns=cols_to_drop_eval, errors='ignore')
w_valid = valid_data['clase_peso']
X_test = test_data.drop(columns=cols_to_drop_eval, errors='ignore')

train_data_initial = data[data['foto_mes'].isin(meses_train)]
X_train_initial = train_data_initial.drop(columns=cols_to_drop_eval, errors='ignore')
y_train_initial_bin1 = train_data_initial['clase_binaria1']
y_train_initial_bin2 = train_data_initial['clase_binaria2']
w_train_initial = train_data_initial['clase_peso']

meses_completos = meses_train + [mes_valid]
train_valid_data = data[data['foto_mes'].isin(meses_completos)]
X_train_valid = train_valid_data.drop(columns=cols_to_drop_eval, errors='ignore')
y_train_valid_bin1 = train_valid_data['clase_binaria1']
y_train_valid_bin2 = train_valid_data['clase_binaria2']
w_train_valid = train_valid_data['clase_peso']

In [ ]:
# Paso 1: 
# Para cada semilla: Entrenar y predecir en abril. Luego re-entrenar con todos los meses y predecir en junio.

print(f"\n{'='*60}")
print("🌱 ENTRENANDO MODELOS Y PREDICIENDO EN ABRIL")
print(f"{'='*60}")

probabilidades_abril_todas_semillas = []
probabilidades_junio_todas_semillas = []
umbrales_individuales = []  # Por si querés comparar
ganancias_por_seed = {}

for i, seed in enumerate(semillas):
    print(f"\n🌱 Semilla {seed} ({i+1}/{len(semillas)})")
    params_seed = best_params.copy()
    params_seed['seed'] = seed

    # --- ENTRENAR CON MESES DE TRAIN (Ene+Feb+Mar) ---
    y_final = y_train_initial_bin1 if best_target == "binaria1" else y_train_initial_bin2
    train_dataset = lgb.Dataset(X_train_initial, label=y_final, weight=w_train_initial)
    
    model_abril = lgb.train(params_seed, train_dataset, num_boost_round=best_iter)
    
    # --- PREDECIR EN ABRIL ---
    y_pred_abril = model_abril.predict(X_valid)
    probabilidades_abril_todas_semillas.append(y_pred_abril)
    
    # Guardar umbral individual para comparar
    umbral_ind, N_ind, gan_ind, gan_curve = mejor_umbral_probabilidad(y_pred_abril, w_valid)
    umbrales_individuales.append(umbral_ind)
    ganancias_por_seed[seed] = gan_curve
    print(f"   📊 Umbral individual: {umbral_ind:.6f}, N={N_ind}, Ganancia=${gan_ind:,.0f}")
    
    # --- RE-ENTRENAR CON TODOS LOS MESES (Ene+Feb+Mar+Abril) ---
    y_train_valid = y_train_valid_bin1 if best_target == "binaria1" else y_train_valid_bin2
    train_data_combined = lgb.Dataset(X_train_valid, label=y_train_valid, weight=w_train_valid)
    model_final = lgb.train(params_seed, train_data_combined, num_boost_round=best_iter)
    
    # --- PREDECIR EN JUNIO ---
    y_pred_junio = model_final.predict(X_test)
    probabilidades_junio_todas_semillas.append(y_pred_junio)
    print(f"   ✅ Predicciones de abril y junio guardadas")


🌱 ENTRENANDO MODELOS Y PREDICIENDO EN ABRIL

🌱 Semilla 181459 (1/10)
   📊 Umbral individual: 0.028007, N=8072, Ganancia=$354,140,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 306491 (2/10)
   📊 Umbral individual: 0.018153, N=10718, Ganancia=$355,820,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 336251 (3/10)
   📊 Umbral individual: 0.015892, N=11592, Ganancia=$357,840,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 900577 (4/10)
   📊 Umbral individual: 0.016319, N=11506, Ganancia=$357,220,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 901751 (5/10)
   📊 Umbral individual: 0.023214, N=9106, Ganancia=$358,420,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 182009 (6/10)
   📊 Umbral individual: 0.020418, N=9845, Ganancia=$357,680,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 182011 (7/10)
   📊 Umbral individual: 0.022290, N=9412, Ganancia=$360,100,000
   ✅ Predicciones de abril y junio guardadas

🌱 Semilla 182

In [ ]:
# Paso 2: Crear ENSEMBLE de abril (promediar probabilidades predichas) y encontrar umbral óptimo que maximiza la ganancia con la funcion mejor_umbral_probabilidad

print(f"\n{'='*60}")
print("🎯 CREANDO ENSEMBLE DE ABRIL Y OPTIMIZANDO UMBRAL")
print(f"{'='*60}")

# Promediar probabilidades de abril entre todas las semillas
matriz_abril = np.array(probabilidades_abril_todas_semillas)  # shape: (n_semillas, n_registros)
probabilidades_abril_ensemble = np.mean(matriz_abril, axis=0)

print(f"📊 Ensemble de abril creado:")
print(f"   - Forma de matriz: {matriz_abril.shape}")
print(f"   - Probabilidades promedio: {probabilidades_abril_ensemble.shape}")

# Encontrar umbral óptimo DEL ENSEMBLE
umbral_ensemble, N_ensemble, ganancia_ensemble, curva_ensemble = mejor_umbral_probabilidad(
    probabilidades_abril_ensemble, 
    w_valid
)

print(f"\n✅ UMBRAL ÓPTIMO DEL ENSEMBLE: {umbral_ensemble:.6f}")
print(f"   📊 N en ese umbral: {N_ensemble}")
print(f"   💰 Ganancia máxima: ${ganancia_ensemble:,.0f}")

# Comparar con umbral promedio de los individuales
umbral_promedio_individual = np.mean(umbrales_individuales)
print(f"\n📊 COMPARACIÓN:")
print(f"   Umbral del ensemble:        {umbral_ensemble:.6f}")
print(f"   Umbral promedio individual: {umbral_promedio_individual:.6f}")
print(f"   Diferencia:                 {abs(umbral_ensemble - umbral_promedio_individual):.6f}")
print(f"   Desv. estándar individuales: {np.std(umbrales_individuales):.6f}")


🎯 CREANDO ENSEMBLE DE ABRIL Y OPTIMIZANDO UMBRAL
📊 Ensemble de abril creado:
   - Forma de matriz: (10, 163418)
   - Probabilidades promedio: (163418,)

✅ UMBRAL ÓPTIMO DEL ENSEMBLE: 0.021360
   📊 N en ese umbral: 9757
   💰 Ganancia máxima: $361,780,000

📊 COMPARACIÓN:
   Umbral del ensemble:        0.021360
   Umbral promedio individual: 0.019746
   Diferencia:                 0.001614
   Desv. estándar individuales: 0.003891


In [ ]:
umbral_ensemble # umbral óptimo del ensemble del mes de abril

0.021360187124262603

In [ ]:
umbral_junio = umbral_ensemble * 0.82 # ajusto a mano el umbral para junio guiandome por la tendencia de los targets los meses previos

In [ ]:
# Paso 3: Crear ENSEMBLE de junio y aplicar umbral

print(f"\n{'='*60}")
print("🚀 CREANDO ENSEMBLE DE JUNIO Y APLICANDO UMBRAL")
print(f"{'='*60}")

# Promediar probabilidades de junio entre todas las semillas
matriz_junio = np.array(probabilidades_junio_todas_semillas)
probabilidades_junio_ensemble = np.mean(matriz_junio, axis=0)

print(f"📊 Ensemble de junio creado:")
print(f"   - Forma de matriz: {matriz_junio.shape}")
print(f"   - Probabilidades promedio: {probabilidades_junio_ensemble.shape}")
print(f"   - Min prob: {probabilidades_junio_ensemble.min():.6f}")
print(f"   - Max prob: {probabilidades_junio_ensemble.max():.6f}")
print(f"   - Media prob: {probabilidades_junio_ensemble.mean():.6f}")

# Aplicar umbral
prediccion_final_binaria = (probabilidades_junio_ensemble >= umbral_junio).astype(int)
N_enviados_final = prediccion_final_binaria.sum()

print(f"\n✅ PREDICCIÓN FINAL CON ENSEMBLE:")
print(f"   🎯 Umbral usado: {umbral_junio:.6f}")  
print(f"   📮 Clientes marcados: {N_enviados_final:,}")
print(f"   📊 Proporción de positivos: {N_enviados_final/len(prediccion_final_binaria)*100:.2f}%")


resultado_ensemble = {
    'umbral_optimo_ensemble': umbral_ensemble,
    'umbral_usado_junio': umbral_junio,  
    'N_en_umbral': N_ensemble,
    'ganancia_maxima_abril': ganancia_ensemble,
    'umbrales_individuales': umbrales_individuales,
    'umbral_promedio_individual': umbral_promedio_individual,
    'probabilidades_abril_ensemble': probabilidades_abril_ensemble,
    'probabilidades_junio_ensemble': probabilidades_junio_ensemble,
    'prediccion_binaria': prediccion_final_binaria,
    'N_enviados': N_enviados_final,
    'curva_ganancia': curva_ensemble
}


🚀 CREANDO ENSEMBLE DE JUNIO Y APLICANDO UMBRAL
📊 Ensemble de junio creado:
   - Forma de matriz: (10, 164313)
   - Probabilidades promedio: (164313,)
   - Min prob: 0.000003
   - Max prob: 0.972633
   - Media prob: 0.008155

✅ PREDICCIÓN FINAL CON ENSEMBLE:
   🎯 Umbral usado: 0.017515
   📮 Clientes marcados: 11,988
   📊 Proporción de positivos: 7.30%


In [ ]:
# Archivo para subir a Kaggle - ENSEMBLE


print(f"\n{'='*60}")
print(f"📦 GENERANDO SUBMISSION CON ENSEMBLE")
print(f"{'='*60}")
print(f"🎯 Umbral usado para junio: {umbral_junio:.6f}")  
print(f"📮 Número de envíos: {N_enviados_final:,}")

submission = pd.DataFrame({
    'numero_de_cliente': test_data['numero_de_cliente'].values,
    'Predicted': prediccion_final_binaria
})

submission_filename = f'{NOMBRE_NOTEBOOK}_TRIAL{trial_number}_ENSEMBLE_U{umbral_junio:.6f}_N{N_enviados_final}.csv'  
submission.to_csv(submission_filename, index=False)

print(f"\n💾 Archivo guardado: {submission_filename}")
print(f"📊 Distribución: {submission['Predicted'].value_counts().to_dict()}")
print(f"   - Clase 0 (no enviar): {(prediccion_final_binaria == 0).sum():,}")
print(f"   - Clase 1 (enviar):    {(prediccion_final_binaria == 1).sum():,}")
print(f"   - Total registros:     {len(prediccion_final_binaria):,}")

print("\n📋 Muestra del submission:")
print(submission.head(10))

# Estadísticas completas del ensemble
print(f"\n{'='*60}")
print(f"📊 RESUMEN COMPLETO DEL ENSEMBLE")
print(f"{'='*60}")
print(f"🌱 Número de semillas usadas: {len(semillas)}")
print(f"   Semillas: {semillas}")

print(f"\n📐 Umbrales individuales (por semilla):")
for i, (seed, umbral) in enumerate(zip(semillas, umbrales_individuales), 1):
    print(f"   {i}. Seed {seed}: {umbral:.6f}")

print(f"\n🎯 Umbrales finales:")
print(f"   Promedio de individuales:     {umbral_promedio_individual:.6f}")
print(f"   Umbral óptimo en Abril:       {umbral_ensemble:.6f}")  # ✅ Cambiado texto
print(f"   Umbral usado en Junio:        {umbral_junio:.6f} ← USADO")  # ✅ Agregado
if umbral_junio != umbral_ensemble:  # ✅ Agregado
    print(f"   Ajuste aplicado:              {((umbral_junio - umbral_ensemble) / umbral_ensemble) * 100:+.2f}%")
print(f"   Desviación estándar:          {np.std(umbrales_individuales):.6f}")

print(f"\n💰 Ganancia en validación (Abril):")
print(f"   Ganancia máxima del ensemble: ${ganancia_ensemble:,.0f}")
print(f"   N óptimo en abril: {N_ensemble:,}")

print(f"\n📊 Probabilidades del ensemble (Junio):")
print(f"   - Mínima:  {probabilidades_junio_ensemble.min():.6f}")
print(f"   - Máxima:  {probabilidades_junio_ensemble.max():.6f}")
print(f"   - Media:   {probabilidades_junio_ensemble.mean():.6f}")
print(f"   - Mediana: {np.median(probabilidades_junio_ensemble):.6f}")
print(f"   - Q1:      {np.percentile(probabilidades_junio_ensemble, 25):.6f}")
print(f"   - Q3:      {np.percentile(probabilidades_junio_ensemble, 75):.6f}")

print(f"\n📮 Predicción final:")
print(f"   Clientes a contactar: {N_enviados_final:,} ({N_enviados_final/len(prediccion_final_binaria)*100:.2f}%)")
print(f"   Clientes sin contactar: {(prediccion_final_binaria == 0).sum():,}")

print(f"\n✅ Listo para subir a Kaggle: {submission_filename}")
print(f"{'='*60}")


📦 GENERANDO SUBMISSION CON ENSEMBLE
🎯 Umbral usado para junio: 0.017515
📮 Número de envíos: 11,988

💾 Archivo guardado: CV_clasico_semillas_shock_aguinaldo_TRIAL113_ENSEMBLE_U0.017515_N11988.csv
📊 Distribución: {0: 152325, 1: 11988}
   - Clase 0 (no enviar): 152,325
   - Clase 1 (enviar):    11,988
   - Total registros:     164,313

📋 Muestra del submission:
   numero_de_cliente  Predicted
0          249320580          0
1          249368642          0
2          249483501          0
3          249493416          0
4          249494219          0
5          249580854          0
6          249654219          1
7          249693963          0
8          249739113          0
9          249914502          0

📊 RESUMEN COMPLETO DEL ENSEMBLE
🌱 Número de semillas usadas: 10
   Semillas: [181459, 306491, 336251, 900577, 901751, 182009, 182011, 182027, 182029, 182041]

📐 Umbrales individuales (por semilla):
   1. Seed 181459: 0.028007
   2. Seed 306491: 0.018153
   3. Seed 336251: 0.015892
   